In [1]:
#!pip install -U skorch
!pip install efficientnet_pytorch

In [2]:
import os
import shutil
import hashlib

import numpy as np
import pandas as pd
import cv2

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from PIL import Image
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import albumentations as A
from albumentations.pytorch import ToTensor

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
DIR_IMAGES = "/content/drive/My Drive/train_data/images/"
DIR_DF = "/content/drive/My Drive/train_data/train.csv"

SIZE = 224
BATCH_SIZE = 64
SEED = 42
MODEL_NAME = 'efficientnet-b0'

### Check the amount of data for each class


In [6]:
train_data = pd.read_csv('/content/drive/My Drive/train_data/train.csv')
train_data['emergency_or_not'].value_counts()

0    965
1    681
Name: emergency_or_not, dtype: int64

In [7]:
train_data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


### Create Image metadata

In [8]:
def calculate_hash(im):
    md5 = hashlib.md5()
    md5.update(np.array(im).tostring())
    
    return md5.hexdigest()


def get_image_meta(image_id, image_src, dataset='train'):
    im = Image.open(image_src)
    extrema = im.getextrema()

    meta = {
        'image_id': image_id,
        'dataset': dataset,
        'hash': calculate_hash(im),
        'r_min': extrema[0][0],
        'r_max': extrema[0][1],
        'g_min': extrema[1][0],
        'g_max': extrema[1][1],
        'b_min': extrema[2][0],
        'b_max': extrema[2][1],
        'height': im.size[0],
        'width': im.size[1],
        'format': im.format,
        'mode': im.mode
    }
    return meta

### Dataset Class

In [23]:
class VehicleDataset(Dataset):
  """ Emergency Vehicles Dataset. """
  def __init__(self, dataframe, root_dir, transform=None, train=True):
    """ 
    Parameters:
      csv_file(string): Path to the csv file containing the labels.
      root_dir(string): Path to the folder that contains the images.
      transforms(callable): Optional transforms to be applied on a sample."""
    self.vehicles_frame = dataframe
    self.root_dir = root_dir
    self.transform = transform
    self.train = train

  def __len__(self):
    return(self.vehicles_frame.shape[0])
  
  def __getitem__(self, idx):
    img_name = self.root_dir + self.vehicles_frame['image_names'].iloc[idx]
    image = cv2.imread(img_name, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transformed = self.transform(image=image)
    image = transformed['image']
    if(self.train):
      labels = torch.tensor(self.vehicles_frame['emergency_or_not'].iloc[idx])
    else:
      labels = idx
    return(image, labels)

### Define Transforms

In [24]:
train_transform = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(rotate_limit=0.5, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),
    
    A.Normalize(p=1.0),
    ToTensor(),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensor(),
])

### Test Dataset Class



In [11]:
# train_dataset = VehicleDataset(train_data, DIR_IMAGES, train_transform)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# images, labels = next(iter(train_dataloader))
# del train_dataloader, train_dataset

###  V4 - EfficientNet Model


In [58]:
model = models.resnet50(pretrained=True)
ct = 0
for name, child in model.named_children():
    ct += 1
    if (ct < 8):
      print(name)
      for name2, params in child.named_parameters():
        params.requires_grad = False

conv1
bn1
relu
maxpool
layer1
layer2
layer3


In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.fc = nn.Sequential(nn.Linear(2048, 256),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.5),
                                 nn.Linear(256, 2),
                                 nn.Softmax(dim=1))
PATH = 'init.pth'
torch.save(model.state_dict(), PATH)
model.to(device);

In [61]:
def compute_val_metrics(model, valid_loader):
  model.eval()
  validation_loss = 0
  correct = 0
  size = 0
  with torch.no_grad():
    for inputs, labels in valid_loader:
      # Move data to the Cuda.
      inputs, labels = inputs.to(device), labels.float().to(device)
      # Compute the model outputs
      output= model.forward(inputs)
      # Compute loss function
      loss = criterion(output[:, -1], labels)
      validation_loss += loss
      # Compute Result
      result = output.max(axis=1)[1]
      # Find the number of correctly classifid examples
      correct += torch.sum(torch.eq(result.type(labels.type()), labels)).item()
      # Find the total number of samples
      size += labels.size(0)
  return(validation_loss, (correct * 100 /size))


def train_one_epoch(model, optmizer, train_loader, valid_loader):
  train_loss = 0
  model.train()
  for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels.float().to(device)

    optimizer.zero_grad()
  
    logps = model.forward(inputs)
    loss = criterion(logps[:, -1], labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  validation_loss, valid_accuracy = compute_val_metrics(model, valid_loader)
  _, train_accuracy = compute_val_metrics(model, train_loader)
  return(train_loss, validation_loss, valid_accuracy, train_accuracy)

In [62]:
folds = KFold(n_splits=5, shuffle=True, random_state=SEED)

In [63]:
from collections import defaultdict


for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_data)):
  epochs = 10
  # Dicts to store training and test accuracy

  # Validation DataFrame
  valid = train_data.iloc[valid_idx]
  valid.reset_index(drop=True, inplace=True)
  # Training DataFrame
  train = train_data.iloc[train_idx]
  train.reset_index(drop=True, inplace=True)
  # Train DataLoader
  train_dataset = VehicleDataset(train, DIR_IMAGES, train_transform)
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  # Valid Dataloder
  valid_dataset = VehicleDataset(valid, DIR_IMAGES, transforms_valid)
  valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
  # Reset model and loss functions every iteration
  criterion = nn.BCELoss()
  model.load_state_dict(torch.load(PATH)) 
  model.to(device);
  parameters =  list(model.layer4.parameters()) + list(model.fc.parameters())
  optimizer = optim.Adam(parameters, lr=0.0005, weight_decay=)
  for epoch in range(epochs):
    print(f"Fold {i_fold}")
    train_loss, valid_loss, valid_accuracy, train_accuracy = train_one_epoch(model, optimizer, train_dataloader, valid_dataloader)

    print(f"Epoch {epoch+1}/{epochs}.. "
            f"Train loss: {train_loss:.3f}.. "
            f"Val Loss:{valid_loss:.3f}.. "
            f"Val Accuracy:{valid_accuracy}.."
            f"Train Accuracy:{train_accuracy}")

Fold 0
Epoch 1/10.. Train loss: 7.360.. Val Loss:3.109.. Val Accuracy:90.9090909090909..Train Accuracy:88.29787234042553
Fold 0
Epoch 2/10.. Train loss: 4.644.. Val Loss:1.277.. Val Accuracy:95.15151515151516..Train Accuracy:94.30091185410335
Fold 0
Epoch 3/10.. Train loss: 4.268.. Val Loss:1.070.. Val Accuracy:95.45454545454545..Train Accuracy:95.13677811550151
Fold 0
Epoch 4/10.. Train loss: 3.110.. Val Loss:0.713.. Val Accuracy:96.06060606060606..Train Accuracy:93.46504559270517
Fold 0
Epoch 5/10.. Train loss: 2.951.. Val Loss:0.724.. Val Accuracy:95.75757575757575..Train Accuracy:96.42857142857143
Fold 0
Epoch 6/10.. Train loss: 3.229.. Val Loss:0.881.. Val Accuracy:93.93939393939394..Train Accuracy:96.12462006079028
Fold 0
Epoch 7/10.. Train loss: 2.462.. Val Loss:0.904.. Val Accuracy:96.06060606060606..Train Accuracy:94.98480243161094
Fold 0
Epoch 8/10.. Train loss: 2.272.. Val Loss:0.619.. Val Accuracy:94.24242424242425..Train Accuracy:96.580547112462
Fold 0
Epoch 9/10.. Train l

### Evaluate accuracy on the training set


In [ ]:
test_train_dataset = VehicleDataset(DIR_DF, DIR_IMAGES, train_transform, False)
test_train_dataloader = DataLoader(test_train_dataset, batch_size=64, shuffle=True)
train_data['results'] = 0
model.eval()
with torch.no_grad():
  for image_id, index in test_train_dataloader:
    image_id = image_id.cuda()
    output = model(image_id)
    result = output.max(axis=1)[1].cpu().numpy()
    train_data['results'].iloc[index] = result


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
train_accuracy = (1 - (np.abs(train_data['results'] - train_data['emergency_or_not']).sum() / train_data.shape[0]))
train_accuracy

0.9951397326852977

### Sumbit the model and report test accuracy


In [ ]:
test_df = pd.read_csv('/content/drive/My Drive/train_data/test_vc2kHdQ.csv')
test_df['emergency_or_not'] = 0
test_dataset = VehicleDataset('/content/drive/My Drive/train_data/test_vc2kHdQ.csv', DIR_IMAGES, transforms_valid, False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model.eval()
with torch.no_grad():
  for image_id, index in test_dataloader:
    image_id = image_id.cuda()
    output = model(image_id)
    result = output.max(axis=1)[1].cpu().numpy()
    test_df['emergency_or_not'].iloc[index] = result

In [ ]:
test_df.to_csv('results.csv')